# Market Data Integration Guide

## Overview
This guide covers integrating with TastyTrade's market data API using Python WebSocket implementation, with a focus on asyncio-based approach.

## TastyTrade API Resources
- [Developer Portal](https://developer.tastytrade.com/)
- [Streaming Market Data Documentation](https://developer.tastytrade.com/streaming-market-data/)
- [DXLink Protocol Reference](https://demo.dxfeed.com/dxlink-ws/debug/#/protocol)

## WebSocket Implementation Details

### Selected Library
We'll be using the [websockets](https://websockets.readthedocs.io/en/stable/) library for Python, which provides high-level WebSocket implementation (not to be confused with the lower-level `websocket` library).

### Key Resources
- **Documentation**: Complete guides available on [readthedocs.io](https://websockets.readthedocs.io/en/stable/)
- **Reference Implementation**: [Example using Threading](https://github.com/LordKaT/tastytrade_api_thing/blob/main/lib/TTWebsocket.py) by [LordKaT](https://github.com/LordKaT)

### Technical Approach
The implementation will use `asyncio` due to its comprehensive [feature support](https://websockets.readthedocs.io/en/stable/reference/features.html) in the websockets library.

### AsyncIO Learning Resources
For better understanding of the asyncio implementation:
- [How does async/await work in Python 3.5](https://snarky.ca/how-the-heck-does-async-await-work-in-python-3-5/)
- [Cooperative multitasking with Coroutines](https://pymotw.com/3/asyncio/coroutines.html)
- [A Curious course on Coroutines and Concurrency](http://www.dabeaz.com/coroutines/)

## Implementation Notes
While previous experience has been primarily with traditional threading, this project will explore asyncio-based implementation for its potential benefits in handling WebSocket connections.

In [1]:
import logging

import asyncio
from tastytrade.logging import setup_logging
from tastytrade.sessions import Credentials
from tastytrade.sessions.requests import AsyncSessionHandler
from tastytrade.sessions.sockets import WebSocketManager
from tastytrade.sessions.messaging import Channels
from tastytrade.sessions.dxlink import DXLinkClient

from datetime import datetime, timedelta

logging.getLogger().handlers.clear()
logger = logging.getLogger(__name__)
setup_logging(logging.DEBUG)

TEST = True
ENV = "Live"

shutdown = asyncio.Event()

## Test WebSocket context manager

In [ ]:
session = await AsyncSessionHandler.create(Credentials(env=ENV))

today = datetime.now().strftime("%y%m%d")
next_day = datetime.now() + timedelta(days=1)
while next_day.weekday() >= 5:  # 5 = Saturday, 6 = Sunday
    next_day += timedelta(days=1)
next_business_day = next_day.strftime("%y%m%d")

channel = Channels.Quotes
symbols = [
    "SPX",
    "NVDA",
    "BTC/USD:CXTALP",
    f".SPXW{next_business_day}P5990",
    f".SPXW{next_business_day}P5995",
    f".SPXW{next_business_day}P6000",
]

async with WebSocketManager(session) as websocket:
    dxlink_client = DXLinkClient(websocket)
    await dxlink_client.setup_feeds()
    await asyncio.sleep(5)
    await dxlink_client.subscribe_to_feeds(symbols)

    if TEST:
        await asyncio.sleep(20)
    else:
        await websocket.send_keepalives()

    await dxlink_client.queue_manager.cleanup()

await session.close()

## Test individual components

In [ ]:
session = await AsyncSessionHandler.create(Credentials(env=ENV))

exlink = WebSocketManager(session)
await exlink.open()

In [ ]:
dxlink_client = DXLinkClient(exlink)
await dxlink_client.setup_feed(Channels.Quotes)
await dxlink_client.subscribe_to_feed(Channels.Quotes)

In [ ]:
asyncio.all_tasks()

In [ ]:
await exlink.close()

## Old Feed Data

In [ ]:
# from datetime import datetime

# SUBSCRIPTION_REQUEST = {
#     "type": "FEED_SUBSCRIPTION",
#     "reset": True,
#     "add": [
#         {"type": "Trade", "symbol": "BTC/USD:CXTALP"},
#         {"type": "Trade", "symbol": "SPY"},
#         {"type": "Profile", "symbol": "BTC/USD:CXTALP"},
#         {"type": "Profile", "symbol": "SPY"},
#         {"type": "Summary", "symbol": "BTC/USD:CXTALP"},
#         {"type": "Summary", "symbol": "SPY"},
#         {"type": "Greeks", "symbol": f".SPXW{datetime.now().strftime('%y%m%d')}P5990"},
#         {"type": "Greeks", "symbol": f".SPXW{datetime.now().strftime('%y%m%d')}P5980"},
#         {"type": "Greeks", "symbol": f".SPXW{datetime.now().strftime('%y%m%d')}P5970"},
#         {"type": "Quote", "symbol": f".SPXW{datetime.now().strftime('%y%m%d')}P5990"},
#         {"type": "Quote", "symbol": f".SPXW{datetime.now().strftime('%y%m%d')}P5980"},
#         {"type": "Quote", "symbol": f".SPXW{datetime.now().strftime('%y%m%d')}P5970"},
#         {"type": "Quote", "symbol": "SPX"},
#         {"type": "Quote", "symbol": "BTC/USD:CXTALP"},
#     ],
# }